# A09
# Bagging

In [34]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from scipy.stats import mode

In [37]:
default = pd.read_csv("Default.csv")
default["default"] = (default["default"] == "Yes").astype(int)
default["student"] = (default["student"] == "Yes").astype(int)

x_default = default.drop(columns=["default"])
y_default = default["default"]

model_lr = LogisticRegression()
model_lr.fit(x_default, y_default)
pred_lr = model_lr.predict_proba(x_default)[:, 1]
pred_lr_class = (pred_lr > 0.5).astype(int)

auc_lr = roc_auc_score(y_default, pred_lr)
accuracy_lr = accuracy_score(y_default, pred_lr_class)

n_bootstraps = 5000
all_predictions = np.zeros((len(default), n_bootstraps))

feature_names = x_default.columns

for i in range(n_bootstraps):
    cols = np.random.choice(feature_names, size=2, replace=False)
    boot_sample = resample(default, replace=True, n_samples=5000)
    
    x_boot = boot_sample[cols]
    y_boot = boot_sample["default"]
    model = DecisionTreeClassifier()
    model.fit(x_boot, y_boot)
    all_predictions[:, i] = model.predict_proba(x_default[cols])[:, 1]

bagging_pred_mean = all_predictions.mean(axis=1)
auc_bagging = roc_auc_score(y_default, bagging_pred_mean)

bagging_pred_class = (bagging_pred_mean > 0.5).astype(int)
accuracy_bagging = accuracy_score(y_default, bagging_pred_class)
accuracy_bagging = (bagging_pred_class == y_default).mean()


In [39]:
pd.DataFrame({
    "Model": ["Logistic Regression", "Bagging con Árboles de Decisión"],
    "AUC": [auc_lr, auc_bagging],
    "Accuracy": [accuracy_lr, accuracy_bagging]
})

,Model,AUC,Accuracy
0,Logistic Regression,0.949571,0.9732
1,Bagging con Árboles de Decisión,0.999963,0.9843


Los resultados comparan el desempeño del modelo de Regresión Logística contra el método de Bagging. La regresión logística obtuvo un AUC de aproximadamente 0.95 y una exactitud cercana a 0.973, lo cual indica un desempeño sólido. Sin embargo, el modelo de Bagging mostró valores extremadamente altos, con un AUC prácticamente perfecto (0.99996) y una exactitud de 0.984. Aunque en apariencia Bagging supera a la regresión logística, un AUC tan cercano a 1 suele ser señal de sobreajuste, especialmente si las predicciones se evaluaron sobre los mismos datos usados para entrenar los árboles. Por tanto, aunque Bagging parece más preciso, es importante validar estos resultados en datos nuevos o mediante técnicas como out-of-bag para confirmar que el modelo generaliza bien.